In [1]:
env = "production"
env = "staging"


# s3_bucket = "s3://mist-aggregated-stats-{env}/entity_event/entity_event-{env}/".format(env=env)
date_day = "2020-03-1*"
hr = '*'

# s3_capacity_path = "dt={day}/hr={hr}/CapacityAnomalyEvent_*.seq".format(day=date_day, hr=hr)
# s3_coverage_path = "dt={day}/hr={hr}/CoverageAnomalyEvent_*.seq".format(day=date_day, hr=hr)
# s3_capacity_path = s3_bucket + s3_capacity_path
# s3_coverage_path = s3_bucket + s3_coverage_path
# print(s3_capacity_path, "\n", s3_coverage_path)

s3_bucket = "s3://mist-secorapp-staging/sle-capacity-anomaly/sle-capacity-anomaly-staging/".format(env=env)
s3_path = s3_bucket + "dt={day}/hr={hr}/*.seq".format(day=date_day, hr=hr)

print(s3_path)


s3://mist-secorapp-staging/sle-capacity-anomaly/sle-capacity-anomaly-staging/dt=2020-03-*/hr=*/*.seq


In [2]:
# rdd_capacity = spark.sparkContext.sequenceFile(s3_capacity_path)
# rdd_coverage = spark.sparkContext.sequenceFile(s3_coverage_path)
rdd_capacity = spark.sparkContext.sequenceFile(s3_path)

In [3]:
rdd_capacity.first()

(55072,
 bytearray(b'{"radio_5s":1,"util_nonwifi_std_base":5.0,"channel":6,"sle_coverage_base":1.0,"util_nonwifi_deviation":0.02180516335405495,"rssi_mean":-56.8921568627451,"rad_id":"r24","sle_capacity_anomaly_score":0.0,"util_all_mean_base":90.67844744950943,"anomaly_score":2.6348636929746667,"dev":"r0","anomaly_type":"cochannel","band":"24","util_cochannel_std":5.0,"sle_coverage_anomaly":false,"util_cochannel_mean":71.95719744145556,"util_cochannel_std_base":5.0,"events":[],"avg_nclients":1.0,"sle_capacity_error":0.14004173596603214,"interference_anomaly_score":2.6348636929746667,"util_nonwifi_mean_base":18.21812442019059,"util_ap":8.588402182795107,"sle_coverage_error":0.14004173596603214,"util_cochannel_deviation":7.319065813818521,"ap":"5c-5b-35-0e-09-51","rssi_hist":{"-53":62,"-56":306,"-59":244},"error_rate":0.07442706008346021,"sle_coverage":1.0,"user_seconds_util_high":612,"sle_coverage_base_std":0.1,"sle_coverage_anomaly_score":0.0,"interval":62,"impacted_wlans":["ab105bbe-e

In [4]:
import json
# rdd_site = rdd.map(lambda x: json.loads(x[1])).map(lambda x: (x.get("site_id"), x.get("ap_id"))).groupByKey()
# rdd_site = rdd.map(lambda x: json.loads(x[1])).map(lambda x: (x.get("site_id"))).groupBy('site_id')
df_capacity = rdd_capacity.map(lambda x: json.loads(x[1])).toDF() #.map(lambda x: json.loads(x[1])).



/opt/mistsys/ds_incubator/venv/spark-2.4.4-bin-without-hadoop/python/pyspark/sql/session.py:366: UserWarning: Using RDD of dict to inferSchema is deprecated. Use pyspark.sql.Row instead
  warnings.warn("Using RDD of dict to inferSchema is deprecated. "


In [5]:
# df_capacity
df_capacity.printSchema()

root
 |-- anomaly: string (nullable = true)
 |-- anomaly_score: double (nullable = true)
 |-- anomaly_type: string (nullable = true)
 |-- ap: string (nullable = true)
 |-- avg_nclients: double (nullable = true)
 |-- band: string (nullable = true)
 |-- bandwidth: long (nullable = true)
 |-- channel: long (nullable = true)
 |-- dev: string (nullable = true)
 |-- error_rate: double (nullable = true)
 |-- events: array (nullable = true)
 |    |-- element: string (containsNull = true)
 |-- impacted_wlans: array (nullable = true)
 |    |-- element: string (containsNull = true)
 |-- interference_anomaly_score: double (nullable = true)
 |-- interference_type: string (nullable = true)
 |-- interval: long (nullable = true)
 |-- max_nclients: double (nullable = true)
 |-- org: string (nullable = true)
 |-- power: long (nullable = true)
 |-- rad_id: string (nullable = true)
 |-- radio_5s: long (nullable = true)
 |-- rssi_deviation: double (nullable = true)
 |-- rssi_hist: map (nullable = true)
 | 

In [14]:

# df_capacity.select("org", "site",  "anomaly", "anomaly_score", "anomaly_type", "sle_capacity_anomaly", "util_nonwifi_mean", "util_cochannel_mean").show()

df_capacity_g = df_capacity.select("org", "site", "anomaly_type").groupBy("org", "site", "anomaly_type").count()

In [17]:
# df_capacity_g.show()

In [16]:
df_capacity_g.orderBy('count', ascending=False).take(10)

# df_capacity_g.sort('count', ascending=False).show() #


[Row(org='bf8b581c-eadd-42dc-945c-d278c75f73c5', site='d7c802c4-1d3a-4aa5-90fa-afd14ac7e089', anomaly_type='cochannel', count=80),
 Row(org='2dbab92c-7cc4-11e5-ab8b-1258369c38a9', site='6db15e18-bc94-11e5-91a9-1258369c38a9', anomaly_type='cochannel', count=74),
 Row(org='7e7ce140-d3ce-4ac2-9f02-9ff914d5b7fe', site='5ff3241a-83fc-4421-89de-82c2d530adda', anomaly_type='nonwifi', count=71),
 Row(org='828ed216-66ad-4d23-96ea-b7334645b378', site='8a1c65c6-1962-477f-914e-caa80bf79130', anomaly_type='cochannel', count=30),
 Row(org='6748cfa6-4e12-11e6-9188-0242ac110007', site='67970e46-4e12-11e6-9188-0242ac110007', anomaly_type='nonwifi', count=25),
 Row(org='7e7ce140-d3ce-4ac2-9f02-9ff914d5b7fe', site='c943d077-1b67-4915-a690-e698543d2790', anomaly_type='cochannel', count=24),
 Row(org='6748cfa6-4e12-11e6-9188-0242ac110007', site='67970e46-4e12-11e6-9188-0242ac110007', anomaly_type='cochannel', count=23),
 Row(org='c080ce4d-4e35-4373-bdc4-08df15d257f5', site='27b9fa9e-365e-4d2c-a160-9be78111

In [6]:
# # rdd_site.take(1)
# rdd_capacity_by_site= rdd_capacity.map(lambda x: json.loads(x[1])).map(lambda x: (x.get("site_id"),1)).countByKey()

# rdd_coverage_by_site= rdd_coverage.map(lambda x: json.loads(x[1])).map(lambda x: (x.get("site_id"),1)).countByKey()

# site_capacity_sorted = sorted(rdd_capacity_by_site.items(), key=lambda v:v[1], reverse=True) #, key=lambda(k, v): v)
# site_coverage_sorted = sorted(rdd_coverage_by_site.items(), key=lambda v:v[1], reverse=True) #, key=lambda(k, v): v)


# site_capacity_sorted[:10]


In [18]:
# site_coverage_sorted[:10]

In [10]:
# rdd_coverage_by_site.sort()